## Assignment 1

In [19]:
import numpy as np
import random

def initialize_population(num_cities, population_size):
    return [random.sample(range(num_cities), num_cities) for _ in range(population_size)]

def calculate_distance(path, graph):
    return sum(graph[path[i], path[(i + 1) % len(path)]] for i in range(len(path)))

def fitness(path, graph):
    return 1 / calculate_distance(path, graph)

def select_parents(population, graph, num_parents):
    fitness_scores = [fitness(p, graph) for p in population]
    return list(np.array(population)[np.argsort(fitness_scores)[-num_parents:]])

def crossover(parent1, parent2, num_cities):
    start, end = sorted(random.sample(range(num_cities), 2))
    child = [None] * num_cities
    child[start:end] = parent1[start:end]

    parent2_filtered = [city for city in parent2 if city not in child]
    child = [parent2_filtered.pop(0) if city is None else city for city in child]
    return child

def mutate(path, mutation_rate, num_cities):
    path = path.copy()
    for _ in range(num_cities):
        if random.random() < mutation_rate:
            i, j = random.sample(range(num_cities), 2)
            path[i], path[j] = path[j], path[i]
    return path

def genetic_algorithm(graph, population_size, num_generations, mutation_rate):
    num_cities = len(graph)
    population = initialize_population(num_cities, population_size)
    for generation in range(num_generations):
        new_population = []
        parents = select_parents(population, graph, 2)
        for _ in range(population_size):
            child = crossover(parents[0], parents[1], num_cities)
            child = mutate(child, mutation_rate, num_cities)
            new_population.append(child)
        population = new_population

    best_path = max(population, key=lambda p: fitness(p, graph))
    return best_path, calculate_distance(best_path, graph)

In [28]:
# Example usage
num_cities = 10
graph = np.random.rand(num_cities, num_cities)  # Replace with graph
graph = (graph + graph.T) / 2  # Making the graph symmetric
np.fill_diagonal(graph, np.inf)  # No self-loops

best_path, distance = genetic_algorithm(graph, 100, 500, 0.01)
print(f"Best path: {best_path}\nDistance: {distance}")

Best path: [7, 1, 3, 0, 4, 6, 8, 5, 9, 2]
Distance: 2.918052065857162


## Assignment 2

In [23]:
import numpy as np
import random
import math

def calculate_distance(path, graph):
    return sum(graph[path[i], path[(i + 1) % len(path)]] for i in range(len(path)))

def create_initial_solution(num_cities):
    path = list(range(num_cities))
    random.shuffle(path)
    return path

def get_neighbor(path):
    new_path = path.copy()
    i, j = random.sample(range(len(path)), 2)
    new_path[i], new_path[j] = new_path[j], new_path[i]
    return new_path

def acceptance_probability(old_cost, new_cost, temperature):
    if new_cost < old_cost:
        return 1.0
    return math.exp((old_cost - new_cost) / temperature)

def simulated_annealing(graph, initial_temperature, cooling_rate):
    current_solution = create_initial_solution(len(graph))
    current_cost = calculate_distance(current_solution, graph)
    temperature = initial_temperature

    while temperature > 1:
        neighbor = get_neighbor(current_solution)
        neighbor_cost = calculate_distance(neighbor, graph)

        if acceptance_probability(current_cost, neighbor_cost, temperature) > random.random():
            current_solution, current_cost = neighbor, neighbor_cost

        temperature *= cooling_rate

    return current_solution, current_cost

In [29]:
# Example usage
num_cities = 10
graph = np.random.rand(num_cities, num_cities)  # Replace with graph
graph = (graph + graph.T) / 2  # Making the graph symmetric
np.fill_diagonal(graph, np.inf)  # No self-loops

best_path, distance = simulated_annealing(graph, 10000, 0.9999)
print(f"Best path: {best_path}\nDistance: {distance}")

Best path: [4, 8, 6, 3, 7, 0, 9, 2, 5, 1]
Distance: 4.244807925906157
